<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_PyTorch/blob/main/LinearNeuralNetworkForRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn

In [27]:
class LinearRegressionScratch(nn.Module):
    """The linear regression model implemented from scratch."""
    def __init__(self, num_inputs, lr, sigma=0.01):
        super().__init__()
        self.lr = lr # Store lr as an instance attribute
        self.w = torch.normal(0, sigma, (num_inputs, 1), requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)

    def forward(self, X):
      return torch.matmul(X, self.w) + self.b

    def loss(self, y_hat, y):
      l = (y_hat-y)**2 / 2
      return l.mean()

    def configure_optimizers(self):
      return SGD([self.w, self.b], self.lr)

    def prepare_batch(self, batch):
      return batch

In [42]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

# 1. Define the true weight vector w and bias b
true_w = torch.tensor([2, -3.4]).reshape(-1, 1)
true_b = torch.tensor([4.2])

# Define dataset parameters
num_samples = 1000
num_features = 2
batch_size = 32

# 2. Generate a synthetic dataset X of features and y of labels
X = torch.randn(num_samples, num_features)
y = torch.matmul(X, true_w) + true_b + torch.randn(num_samples, 1) * 0.01


In [43]:
dataset = TensorDataset(X, y)

# 4. Split the TensorDataset into training and validation sets
train_size = int(0.8 * num_samples)
val_size = num_samples - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 5. Create DataLoader instances for both the training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [30]:
class Trainer:
    def __init__(self, model, train_dataloader, val_dataloader=None, max_epochs=10):
        self.model = model
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.max_epochs = max_epochs
        self.optim = model.configure_optimizers()

    def fit(self):
        for epoch in range(self.max_epochs):
            self.model.train()
            total_train_loss = 0
            for X, y in self.train_dataloader:
                # Forward pass
                y_hat = self.model(X)
                # Calculate loss
                loss = self.model.loss(y_hat, y)

                # Backpropagation
                self.optim.zero_grad()
                loss.backward()
                # Optimization step
                self.optim.step()
                total_train_loss += loss.item()

            avg_train_loss = total_train_loss / len(self.train_dataloader)
            print(f"Epoch {epoch + 1}/{self.max_epochs}, Training Loss: {avg_train_loss:.4f}")

            # Validation loop (optional)
            if self.val_dataloader is not None:
                self.model.eval()
                total_val_loss = 0
                with torch.no_grad():
                    for X_val, y_val in self.val_dataloader:
                        y_hat_val = self.model(X_val)
                        val_loss = self.model.loss(y_hat_val, y_val)
                        total_val_loss += val_loss.item()
                avg_val_loss = total_val_loss / len(self.val_dataloader)
                print(f"Epoch {epoch + 1}/{self.max_epochs}, Validation Loss: {avg_val_loss:.4f}")

In [44]:
model = LinearRegressionScratch(num_features, lr=0.03)

In [45]:
max_epochs = 10
trainer = Trainer(model, train_dataloader, val_dataloader, max_epochs=max_epochs)

In [46]:
trainer.fit()
print("Model training initiated.")

Epoch 1/10, Training Loss: 8.6898
Epoch 1/10, Validation Loss: 4.5307
Epoch 2/10, Training Loss: 1.9897
Epoch 2/10, Validation Loss: 1.0820
Epoch 3/10, Training Loss: 0.4675
Epoch 3/10, Validation Loss: 0.2632
Epoch 4/10, Training Loss: 0.1127
Epoch 4/10, Validation Loss: 0.0649
Epoch 5/10, Training Loss: 0.0277
Epoch 5/10, Validation Loss: 0.0162
Epoch 6/10, Training Loss: 0.0069
Epoch 6/10, Validation Loss: 0.0040
Epoch 7/10, Training Loss: 0.0018
Epoch 7/10, Validation Loss: 0.0010
Epoch 8/10, Training Loss: 0.0005
Epoch 8/10, Validation Loss: 0.0003
Epoch 9/10, Training Loss: 0.0002
Epoch 9/10, Validation Loss: 0.0001
Epoch 10/10, Training Loss: 0.0001
Epoch 10/10, Validation Loss: 0.0001
Model training initiated.


In [34]:
class SGD():
    """Minibatch stochastic gradient descent."""
    def __init__(self, params, lr):
        self.params = params
        self.lr = lr

    def step(self):
        for param in self.params:
            # Ensure param.grad is not None before performing operation
            if param.grad is not None:
                param.data -= self.lr * param.grad

    def zero_grad(self):
        for param in self.params:
            if param.grad is not None:
                param.grad.zero_()